# 2022-02-03c Planning a new round of ARC4 combined fits

Results below are for commit: https://github.com/gmarkkula/COMMOTIONSFramework/commit/4ea0ae6c5cdc9bf020f59298af15c0c4cc83da84

(Rerunning the code below for other commits is not likely/guaranteed to give the same results.)

Based on the 2022-01-25 diary notes I should adjust my time estimates for these fits down by about 50 % for the `oVAa*` fits (to about 3 s per parameterisation) and by about 30 % for the `oVA*` fits (to about 4.5 s per parameterisation).

Redoing the same analysis below as I did in the 2022-01-19e diary notes, but with adjusted time estimates as just mentioned.

Creating/updating bash scripts to reflect the estimates below.

In [2]:
# append root folder of repo to PYTHONPATH (https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder)
import sys
from pathlib import Path
parent_path = str(Path('.').absolute().parent)
if not (parent_path in sys.path):
    sys.path.append(parent_path)
    sys.path.append(parent_path + '/SCPaper')
print(sys.path)

import math
import pickle
import sc_fitting

INCL_DET_MODELS = 'all' # ('oVAaoVAloBEvoAI', 'oVAoVAloBEvoAI') # either 'all' or a tuple of names of models to include
EXCL_PROB_MODELS = ('oVAoAN', 'oVAoEAoAN')
MAX_PARAMETERISATIONS = 5000
TIME_PER_PARAMETERISATION_oVAa = 3 # s
TIME_PER_PARAMETERISATION_oVA = 4.5 # s

# load the retained models
with open(sc_fitting.FIT_RESULTS_FOLDER + '/'
          + sc_fitting.RETAINED_DET_FNAME, 'rb') as file_obj:
    det_models = pickle.load(file_obj)
with open(sc_fitting.FIT_RESULTS_FOLDER + '/' 
          + sc_fitting.RETAINED_PROB_FNAME, 'rb') as file_obj:
    prob_models = pickle.load(file_obj)

# loop through all combinations of retained deterministic and 
# probabilistic models
total_fit_time = 0
for det_model in det_models:
    
    det_total_fit_time = 0
    print(f'*** Deterministic model {det_model.model} ***\n')

    # this deterministic model included?
    if INCL_DET_MODELS != 'all':
        assert(type(INCL_DET_MODELS) is tuple)
        if not det_model.model in INCL_DET_MODELS:
            continue
            
    if 'oVAa' in det_model.model:
        time_per_parameterisation = TIME_PER_PARAMETERISATION_oVAa
    else:
        time_per_parameterisation = TIME_PER_PARAMETERISATION_oVA
            
    for prob_model in prob_models:

        # this probabilistic model excluded?
        if prob_model.model in EXCL_PROB_MODELS:
            continue

        # get combined model name
        assert(prob_model.model[0:3] == 'oVA')
        model_name = det_model.model + prob_model.model[3:]

        # get the combined list of parameterisations
        # - get number of parameters and retained parameterisations
        n_det_params = len(det_model.param_names)
        n_prob_params = len(prob_model.param_names)
        n_det_parameterisations = det_model.params_array.shape[0]
        n_prob_parameterisations = prob_model.params_array.shape[0]
        n_comb_parameterisations = (n_det_parameterisations 
                                    * n_prob_parameterisations)
        n_act_comb_parameterisations = min(n_comb_parameterisations, MAX_PARAMETERISATIONS)
        
        print(f'\tModel {model_name}: {n_det_parameterisations}'
              f' x {n_prob_parameterisations} = {n_comb_parameterisations}'
              f' --> {n_act_comb_parameterisations} combined parameterisations...')
        fit_time = n_act_comb_parameterisations * time_per_parameterisation
        print(f'\t\tWill take {fit_time} s ≈ {fit_time / 3600:.1f} h\n')
        det_total_fit_time += fit_time
        total_fit_time += fit_time

    print(f'Total fit time {det_total_fit_time} s ≈ {det_total_fit_time / 3600:.1f} h\n')
        
print(f'\n**********\nGrand total fit time {total_fit_time} s ≈ {total_fit_time / 3600:.1f} h')    

['C:\\GITHUB\\COMMOTIONSFramework\\diary notebooks', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\python39.zip', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\DLLs', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\lib', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64', '', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\lib\\site-packages', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\lib\\site-packages\\win32', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\lib\\site-packages\\win32\\lib', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\lib\\site-packages\\Pythonwin', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\tragma\\.ipython', 'C:\\GITHUB\\COMMOTIONSFramework', 'C:\\GITHUB\\COMMOTIONSFramework/SCPaper']
*** Deterministic model oVAaoVAloBEooBEvoAI ***

	Model oVAaoVAloBEooBEvoAIoEAoSNc: 6807 x 5 = 34035 --> 5000 combined parameterisations...
		Will take 15000 s ≈ 4.2 h

	Model oVAaoVAloBEooBEvoAIoEAoSNv: 6807 x 109 = 